In [33]:
import pandas as pd
import numpy as np
import os
from pprint import pprint

# 1 加载数据

In [34]:
DATA_DIR = './submit/'
files = os.listdir(DATA_DIR)
files = [i for i in files if i[0]!='.']
print(len(files))
pprint(files)

14
['e_ml_79.csv',
 'a_bert_base_8162.csv',
 'p_textrnn_7936.csv',
 'a_bert_base_8048.csv',
 'c_bert_large_8119.csv',
 'c_bert_large_8115.csv',
 'd_robert_base_8098.csv',
 'b_robert_large_8246-1.csv',
 'b_robert_large_8123.csv',
 'h_textcnn_8078.csv',
 'c_bert_large_8171.csv',
 'a_bert_base_8008.csv',
 'b_robert_large_8125.csv',
 'o_fasttext_8070.csv']


In [35]:
train = pd.read_csv('./data/train.csv', sep='\t')
#将标签进行转换
label_id2cate = dict(enumerate(train.categories.unique()))
label_cate2id = {value: key for key, value in label_id2cate.items()}

In [36]:
# 合并所有结果
sub_exp_df = pd.read_csv('./data/sample_submit.csv')
df_merged = sub_exp_df.drop(['categories'], axis=1)
for file in files:
    tmp_df = pd.read_csv(DATA_DIR + file)
    tmp_df['label'] = tmp_df['categories'].map(label_cate2id)
    tmp_df = tmp_df.drop(['categories'], axis=1)
    df_merged = df_merged.merge(tmp_df, how='left', on='paperid')
df_merged.head()

,paperid,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y,label_x,label_y
0,test_00000,13,13,13,13,13,13,13,13,13,13,13,13,13,13
1,test_00001,13,13,13,13,13,13,13,13,13,13,13,13,13,13
2,test_00002,12,6,6,6,6,6,6,6,6,6,6,6,6,6
3,test_00003,14,14,14,14,14,14,14,14,14,14,14,14,14,14
4,test_00004,16,16,16,16,16,16,16,16,16,16,16,16,16,16


# 2 投票融合

In [37]:
# 计票
def work(pres):
    count = [0]*39
    for i in pres:
        count[i] += 1
    out = count.index(max(count))
    return out

In [38]:
tmp_arr = np.array(df_merged.iloc[:,1:])
label_voted = [work(line) for line in tmp_arr]

In [40]:
model_name = "voting-14model-3abc-d-e-h-o-ptest2"
sub_exp_df['categories'] = label_voted
sub_exp_df['categories'] = sub_exp_df['categories'].map(label_id2cate)
sub_exp_df.to_csv('./voting_data/submit_{}.csv'.format(model_name), index=False)

In [ ]:
sub_exp_df.head()

# 3 构造高质量伪标签

In [ ]:
# 计票
def work_high(pres):
    count = [0]*39
    for i in pres:
        count[i] += 1
    p = 11# 该数字根据融合的文件自定义的，如果有11个文件，设该值为11表示，11个结果都投票才可以作为伪标签数据
    if max(count) >p:
        out = count.index(max(count))
    else:
        out = -1
    return out

tmp_arr = np.array(df_merged.iloc[:,1:])
label_voted = [work_high(line) for line in tmp_arr]
# 没有做伪标签的数据数量
print(label_voted.count(-1))

In [ ]:
test_data = train = pd.read_csv('./data/test.csv', sep='\t')
test_data['categories'] = label_voted

In [ ]:
# 删除不能作为伪标签的数据
test_data = test_data.drop(test_data[test_data['categories']==-1].index)
len(test_data)

In [ ]:
# 反编码映射
model_name = "pseudo_label_data"
test_data['categories'] = test_data['categories'].map(label_id2cate)
test_data

In [ ]:
# 读取原始训练集
pseudo_train = pd.read_csv('./data/train.csv',sep="\t")

In [ ]:
# 合并原始训练集和伪标签数据
pseudo_label_train = pd.concat([pseudo_train,test_data])
model_name = "./data/pseudo_train_data"
pseudo_label_train.to_csv('{}.csv'.format(model_name),sep="\t", index=False)